# Hello world
## Optimization algorithms: demo

<em><b>Note</b>: This is just a demo of optimization techniques - not related to any real-life problem.</em>

Imagine the world of strings where the environment demands that the strings are very fit. An unfit string cannot survive. 

How do we define the fitness of a string? The environment demands that the string should be as similar as possible to the target string $s$=''Hello, world''. The closer the string is to ''Hello, world'', the better its chances of survival.

<b> The problem:</b> given a set of random strings of length len($s$) over alphabet $\sigma=\{32 \ldots 122\}$, produce the string which best matches the environment, i.e. with a minimum fitness score. 

Let <em>Weighted Hamming Distance</em> between two strings $x$ and $y$ (both of length $n$) be defined as:<br>
$WH(x,y) = \sum_{i=0}^{n} abs(x[i] - y[i])$

$WH(x,y)$ estimates how far is $x$ from $y$. The lower $WH(x,y)$, the closer string $x$ approaches the target string $y$.<br>
In the space of $\sigma^n$ of all possible strings, we are looking for a global minimum - the string $m$ such that $WH(m,s)$ is minimized. <br>

If we compute $WH(x,s)$ for an arbitrary string $x$, we say that we evaluate the <em>fitness</em> of $x$ with respect to the target string $s$. This is our <em>fitness function</em>. 

## 1. Preparation

In [2]:
# flag which can be turned on/off to print the steps of each optimization
print_steps = True

### 1.1. String fitness function 

In [3]:
s = 'Hello, world'
n = len(s)

# fitness function - weighted hamming distance
string_fitness = lambda x: sum([abs(ord(x[i])- ord(s[i])) for i in range(n)])

### 1.2. Generating initial random strings

In [4]:
import random

alphabet = range(32,123)

def get_random_string(n, sigma):    
    t = [chr(random.choice(sigma)) for i in range(n)]
    return ''.join(t)

# build initial population of many random strings of target length n
def get_rand_population(population_size, n, sigma):
    population=[]
    for i in range(0,population_size):
        individual_str = get_random_string(n, sigma)
        population.append(individual_str)
    return population

# test
print(get_random_string(len(s), alphabet))
print(get_rand_population(3, len(s), alphabet ))

yl$v_[7aKfE2
["'a$+g4TiA&md", 'q,]hHH&)Qehz', 'Sb*agA8gHm`?']


## 2. Random optimize

The simplest possible idea is to try a vaste amount of random solutions and select the one with the best fitness score. Let's see how close we can get to the target $s$=''Hello, world'' with this approach.

After all, isn't evolution just a lottery? ''Physics makes rules, evolution rolls the dice'' ( Charles Cockell. "The Equations of Life").

In [9]:
def random_optimize(population, fitness_function):
    best_score = None
    best_solution = None
    for i in range(len(population)):
        # Calculate fitness 
        score = fitness_function(population[i])

        # Compare it to the best one so far
        if best_score == None or score < best_score:
            best_score = score
            best_solution = population[i]
            if print_steps:
                print(best_solution, " -> score:", best_score)
            
    return (best_score, best_solution, len(population))


# Now the test
string_population = get_rand_population(204800, len(s), alphabet)

best_score, best_sol, iterations = random_optimize (string_population, string_fitness)

FV:Z1VI#Q@0C  -> score: 487
@o)?INe%bgy:  -> score: 432
<k]XCY*\G[^o  -> score: 267
5^e8MD)piPUc  -> score: 223
0f]it*O9huKs  -> score: 217
6hfm?% `VAwb  -> score: 193
0pHkg'EpcpvM  -> score: 176
GOf8p!!Voe\k  -> score: 163
6TeKn"6yveg[  -> score: 144
!ccqhF+ipk^^  -> score: 141
FX_On&#fp]mj  -> score: 113


In [10]:
print()
print("*************Rand optimize**************")
print("trials:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Rand optimize**************
trials:204800, best solution:'FX_On&#fp]mj', best score:113


Randomly trying different solutions is very inefficient because the probability of getting anything close to perfect Hello, world is $(\frac{1}{90})^{12}$. No matter how many random guesses you try – the fitness of the resulting string is very low (the distance from the target remains high).

## 3. Hill climbing

We did not advance far with the random optimization. This happens because each time we come up with some solution and evaluate its fitness - we discard it and try another one - not related to the previous.

The idea behing the <em>hill climbing</em> optimization is to start at a random point in space (choose one random string), and try to move from this point into a direction of better fitness. We will generate a set of neighbors for a randomly selected string, and see if any of these neighbors improve the overall fitness score of the string. 

We continue moving into the direction of this better solution, until there is no more improvement. 

<img src="images/hillclimbing.jpg" style="height:200px;">


The algorithm may produce an optimal solution, but it may also get stuck at the local minimum.
<img src="images/localmin.jpg" style="height:200px;">

In [11]:
def hillclimb_optimize(start_point, fitness_function):
    sol = start_point
    score = fitness_function(sol)
    iterations = 0
    
    # Main loop
    while 1:
        iterations += 1
        
        current_best_score = score
        # Create  neighboring solutions
        neighbors=[]

        for j in range(len(s)):
            # Change character at position j one up or one down
            if ord(sol[j])>32:
                neighbors.append(sol[0:j]+chr(ord(sol[j])-1)+sol[j+1:])
            if ord(sol[j])<122:
                neighbors.append(sol[0:j]+chr(ord(sol[j])+1)+sol[j+1:])
        
        for n_str in neighbors:
            n_score = fitness_function(n_str)
            if n_score < score:
                score = n_score
                sol = n_str
                if print_steps:
                    print(sol, " -> score:", score)      
        
        # If there's no improvement, then we've reached the bottom of the hill
        if score == current_best_score:
            break        
        
        if score == 0:
            break        
        
    return (score, sol, iterations)

# Now the test

rand_str = get_random_string(len(s), alphabet)
best_score, best_sol, iterations = hillclimb_optimize(rand_str, string_fitness)

tKtdA,Q^1ZI+  -> score: 384
sKtdA,Q^1ZI+  -> score: 383
rKtdA,Q^1ZI+  -> score: 382
qKtdA,Q^1ZI+  -> score: 381
pKtdA,Q^1ZI+  -> score: 380
oKtdA,Q^1ZI+  -> score: 379
nKtdA,Q^1ZI+  -> score: 378
mKtdA,Q^1ZI+  -> score: 377
lKtdA,Q^1ZI+  -> score: 376
kKtdA,Q^1ZI+  -> score: 375
jKtdA,Q^1ZI+  -> score: 374
iKtdA,Q^1ZI+  -> score: 373
hKtdA,Q^1ZI+  -> score: 372
gKtdA,Q^1ZI+  -> score: 371
fKtdA,Q^1ZI+  -> score: 370
eKtdA,Q^1ZI+  -> score: 369
dKtdA,Q^1ZI+  -> score: 368
cKtdA,Q^1ZI+  -> score: 367
bKtdA,Q^1ZI+  -> score: 366
aKtdA,Q^1ZI+  -> score: 365
`KtdA,Q^1ZI+  -> score: 364
_KtdA,Q^1ZI+  -> score: 363
^KtdA,Q^1ZI+  -> score: 362
]KtdA,Q^1ZI+  -> score: 361
\KtdA,Q^1ZI+  -> score: 360
[KtdA,Q^1ZI+  -> score: 359
ZKtdA,Q^1ZI+  -> score: 358
YKtdA,Q^1ZI+  -> score: 357
XKtdA,Q^1ZI+  -> score: 356
WKtdA,Q^1ZI+  -> score: 355
VKtdA,Q^1ZI+  -> score: 354
UKtdA,Q^1ZI+  -> score: 353
TKtdA,Q^1ZI+  -> score: 352
SKtdA,Q^1ZI+  -> score: 351
RKtdA,Q^1ZI+  -> score: 350
QKtdA,Q^1ZI+  -> sco

In [12]:
print()
print("*************Hill climbing****************")
print("steps:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Hill climbing****************
steps:385, best solution:'Hello, world', best score:0


## 3. Simulated annealing
The idea of the <em>simulated annealing</em> is borrowed from physics. In metalurgical annealing we heat a metal (alloy) to a very high temperature, the crystal bonds break, and the atoms diffuse more freely. If we cool it slowly, the atoms tend to form more regular crystals producing an alloy with the low termodynamics energy.  

<img style="height:250px;float:right;padding:4px;" src="images/sim_annealing.jpg" >

In the <em>simulated annealing</em> algorithm we set the initial temperature very high, and then we generate a single random neighbor of the current solution. The fitness of this neihgbor can be better or worse that that of the current solution. When the temperature is high, the probability of selecting worse solution is higher. This allows to better explore the search space and get out of the local minimum. The temperature gradually decreases, and so at the end we do not accept worse solutions.

The criterion of accepting ''bad'' solutions:

$p=e^{\frac{-\Delta F}{T}}>R(0,1)$

where $T$ is the current temperature, $R(0,1)$ is a random number between $0$ and $1$, and $\Delta F$ is the difference between the fitness score of new solution and the old solution.

Since the temperature $T$ (the willingness to accept a worse solution) starts very high,
the exponent will be close to 0, and $p$ will almost be 1. As the temperature decreases, the difference between the new fitness score and the old one becomes more important - a bigger difference leads to a lower probability, so the
algorithm will not accept solutions which do not improve fitness - converging to a local minimum after exploring a large global search space.

In [13]:
import math

def annealing_optimize(start_sol, fitness_function, T=10000.0, cool=0.95, step=1):    
    sol = start_sol
    iterations = 0
    
    # Graduate cooling
    while T > 0.01: 
        score = fitness_function(sol)
        
        # Choose one of spots randomly
        i = random.randint(0, len(sol) - 1)

        # Choose rand direction to change the character at position i
        dir = random.randint(-step, step)

        change = ord(sol[i]) + dir
        # out of domain
        if change > 122 or change < 32:
            continue
    
        iterations += 1
        
        # Create a new solution with one of the characters changed
        new_sol = sol[:i] + chr(change) + sol[i+1:]

        # Calculate the new cost
        new_score = fitness_function(new_sol)        

        # Does it make the probability cutoff?
        p = pow(math.e, -(new_score - score) / T)
        
        if new_score < score or p > random.random() :
            sol = new_sol
            score = new_score
            if print_steps:
                print(sol, "-> score:", score)       
        
        if score == 0:
            break
            
        # Decrease the temperature
        T = T * cool
    
    return (score, sol, iterations)

# now test
rand_str = get_random_string(len(s), alphabet)
best_score, best_sol, iterations = annealing_optimize(rand_str, string_fitness,
                                          T=204800.0, cool=0.999, step=1)

04sg5?x$WXF2 -> score: 471
03sg5?x$WXF2 -> score: 472
03sg5?w$WXF2 -> score: 471
03sg5?w$WXG2 -> score: 470
03sg5?w$WXG2 -> score: 470
03sg5@w$WXG2 -> score: 471
03sg5@w#WXG2 -> score: 472
03sf5@w#WXG2 -> score: 473
03sf5@w#VXG2 -> score: 474
02sf5@w#VXG2 -> score: 475
02sf5@w#VXG2 -> score: 475
02sf4@w#VXG2 -> score: 476
02tf4@w#VXG2 -> score: 477
02tf4@w#VXG2 -> score: 477
02tf4@w#VXG2 -> score: 477
02te4@w#VXG2 -> score: 478
02te4@w#VXG2 -> score: 478
02te4@w#VXG2 -> score: 478
02se4@w#VXG2 -> score: 477
02se5@w#VXG2 -> score: 476
02sf5@w#VXG2 -> score: 475
02sf5@w#VXG2 -> score: 475
02sf4@w#VXG2 -> score: 476
02sf4@w#VXG2 -> score: 476
/2sf4@w#VXG2 -> score: 477
/2sf4@x#VXG2 -> score: 478
/2tf4@x#VXG2 -> score: 479
/2tf4@x#VXG2 -> score: 479
/2tf4@x#VXG2 -> score: 479
/2tf4@x"VXG2 -> score: 480
/2tf4@w"VXG2 -> score: 479
/3tf4@w"VXG2 -> score: 478
/3tf4@w"VXG2 -> score: 478
/3tf4@w"VXG3 -> score: 477
/3tf4@w"VXG2 -> score: 478
/3tf4@w!VXG2 -> score: 479
/3tf4@w!VXG2 -> score: 479
/

;.pu3Nw)YYH2 -> score: 473
;.pu2Nw)YYH2 -> score: 474
;.pu2Nw)YYH2 -> score: 474
;.pu2Mw)YYH2 -> score: 473
;/pu2Mw)YYH2 -> score: 472
;/pu2Lw)YYH2 -> score: 471
;/pt2Lw)YYH2 -> score: 470
;/pt2Lw)YZH2 -> score: 469
;.pt2Lw)YZH2 -> score: 470
;.pt2Lw*YZH2 -> score: 469
;.pt2Lw*YZH2 -> score: 469
;.pt2Lw*YZH2 -> score: 469
;.pt2Lw*XZH2 -> score: 470
;.qt2Lw*XZH2 -> score: 471
;.qt2Lw*XZH3 -> score: 470
;.qt2Lw*XZH3 -> score: 470
;.qt3Lw*XZH3 -> score: 469
;.qt4Lw*XZH3 -> score: 468
;.qt4Lw*XZG3 -> score: 469
;/qt4Lw*XZG3 -> score: 468
;/qt4Lv*XZG3 -> score: 467
;/qt4Kv*XZG3 -> score: 466
;/qt4Kv*XZG3 -> score: 466
;/qt4Kv*XZG2 -> score: 467
;.qt4Kv*XZG2 -> score: 468
<.qt4Kv*XZG2 -> score: 467
<.qt4Ku*XZG2 -> score: 466
<.qt4Ku*XZF2 -> score: 467
<.qt4Ku+XZF2 -> score: 466
<.qt4Ku+XZE2 -> score: 467
<.pt4Ku+XZE2 -> score: 466
<.pt4Ku+XZE2 -> score: 466
<.pt4Ku+XZF2 -> score: 465
<.pt4Ku+XZF1 -> score: 466
=.pt4Ku+XZF1 -> score: 465
=.pt4Ku,XZF1 -> score: 464
=.pt4Ku,WZF1 -> score: 465
=

61en7Dz$]SG2 -> score: 468
61en7Dz$]SF2 -> score: 469
61en7Dz$]TF2 -> score: 468
61en7Dz$]TF2 -> score: 468
61en7Dz#]TF2 -> score: 469
71en7Dz#]TF2 -> score: 468
71dn7Dz#]TF2 -> score: 469
71do7Dz#]TF2 -> score: 470
71do7Dy#]TF2 -> score: 469
72do7Dy#]TF2 -> score: 468
72do7Dy#]TF2 -> score: 468
82do7Dy#]TF2 -> score: 467
82do7Dz#]TF2 -> score: 468
92do7Dz#]TF2 -> score: 467
92do7Dz#]TF2 -> score: 467
92do7Ez#]TF2 -> score: 468
92do7Ez#]TF2 -> score: 468
92do7Ez#]TF2 -> score: 468
92do7Ez#]TF2 -> score: 468
92do7Ez"]TF2 -> score: 469
92do7Ez"]TF2 -> score: 469
92do7Ez"]TF2 -> score: 469
92do7Ez"]TF2 -> score: 469
92do7Fz"]TF2 -> score: 470
82do7Fz"]TF2 -> score: 471
81do7Fz"]TF2 -> score: 472
81do7Fz"\TF2 -> score: 473
81co7Fz"\TF2 -> score: 474
81co7Fz"\TE2 -> score: 475
80co7Fz"\TE2 -> score: 476
80co7Fz"\TE1 -> score: 477
80co7Fz"\TE1 -> score: 477
80co7Ez"\TE1 -> score: 476
80bo7Ez"\TE1 -> score: 477
80bo7Ez"\TE1 -> score: 477
80bo7Ez"\TE1 -> score: 477
80bo7Ez"\SE1 -> score: 478
8

H1gt5=w VTE2 -> score: 458
H1gt5>w VTE2 -> score: 459
H1gt5>w VTE2 -> score: 459
I1gt5>w VTE2 -> score: 460
I1gt5>w VTE2 -> score: 460
I1gt5>w VTE2 -> score: 460
H1gt5>w VTE2 -> score: 459
H1gt5>w VTE2 -> score: 459
H1ht5>w VTE2 -> score: 458
H1ht5>w VTE2 -> score: 458
H1ht5>w VTE2 -> score: 458
H1ht5>w VTE2 -> score: 458
H1ht5>w VTE2 -> score: 458
H1it5>w VTE2 -> score: 457
H1it5>w!VTE2 -> score: 456
H1jt5>w!VTE2 -> score: 455
H1it5>w!VTE2 -> score: 456
H1iu5>w!VTE2 -> score: 457
H1iu5>w!VTE2 -> score: 457
H1iu5>w!VSE2 -> score: 458
H1iu5>w!VTE2 -> score: 457
H1iu5>w!WTE2 -> score: 456
H1iu5=w!WTE2 -> score: 455
H1iu5=w!VTE2 -> score: 456
H1ju5=w!VTE2 -> score: 455
H1jt5=w!VTE2 -> score: 454
H1jt5=w!VSE2 -> score: 455
H1jt5=w!WSE2 -> score: 454
H1it5=w!WSE2 -> score: 455
H1it5=w!XSE2 -> score: 454
H1it5=w!XRE2 -> score: 455
H0it5=w!XRE2 -> score: 456
H0it5=w!WRE2 -> score: 457
H0it5=w!WRE2 -> score: 457
H0it5=w"WRE2 -> score: 456
H0it5=w"WSE2 -> score: 455
H0is5=w"WSE2 -> score: 454
H

J/^v47s#Y\@< -> score: 445
J/^v47s#Y\@< -> score: 445
J/^v47t#Y\@< -> score: 446
J/^v47t#Y\@; -> score: 447
J/^v47t#Y\@; -> score: 447
J/^u47t#Y\@; -> score: 446
J/^u47t#Y\@; -> score: 446
J/^u47t#Y]@; -> score: 445
J/^u47t#Y]@; -> score: 445
J/^u47t#Y^@; -> score: 444
J.^u47t#Y^@; -> score: 445
J.^u47s#Y^@; -> score: 444
J._u47s#Y^@; -> score: 443
J-_u47s#Y^@; -> score: 444
J-_u47s#Y^@; -> score: 444
J-_u47s#Y^?; -> score: 445
J-`u47s#Y^?; -> score: 444
J-`u47s#Y^>; -> score: 445
J-`u37s#Y^>; -> score: 446
J-`u38s#Y^>; -> score: 447
J-`u38s#Y^>; -> score: 447
J-`v38s#Y^>; -> score: 448
J-`v38s#Y^>; -> score: 448
J-`v48s#Y^>; -> score: 447
J-`v58s#Y^>; -> score: 446
J-`v58s#Y^>; -> score: 446
J-`v58s#Y^>; -> score: 446
J-`v58s#Y^>; -> score: 446
J-`v58r#Y^>; -> score: 445
J-_v58r#Y^>; -> score: 446
J-_v58r#Y^>; -> score: 446
J-_v58r#Y^=; -> score: 447
J-_v68r#Y^=; -> score: 446
J,_v68r#Y^=; -> score: 447
J,_w68r#Y^=; -> score: 448
J,_w68r#Y^=; -> score: 448
J,_w68r#Y^=; -> score: 448
J

...

I+]d3>n.pO<Z -> score: 405
I+]d3>n.pO<Z -> score: 405
I+]d3>n/pO<Z -> score: 404
I+]d3>n/pO<Z -> score: 404
I+]d3>n/pO=Z -> score: 403
I+]d3>n/pO=Z -> score: 403
I+^d3>n/pO=Z -> score: 402
I+^d3>o/pO=Z -> score: 403
I+^e3>o/pO=Z -> score: 402
I+^d3>o/pO=Z -> score: 403
I+^d3>o/pO=Z -> score: 403
J+^d3>o/pO=Z -> score: 404
J+^d3>o/pO=[ -> score: 403
J+^d3>o/pO=[ -> score: 403
J+^d3>o/pO=[ -> score: 403
J+^d3>o/pO=[ -> score: 403
J+^d3>o/pO=[ -> score: 403
J,^d3>o/pO=[ -> score: 402
J,^d3>o/pN=[ -> score: 403
K,^d3>o/pN=[ -> score: 404
K,^d3>o/oN=[ -> score: 403
K,^d3>o/oN=[ -> score: 403
K,^d3>o0oN=[ -> score: 402
K,^d3>o/oN=[ -> score: 403
K,^d3>o/oN=[ -> score: 403
K,^d3>o/oO=[ -> score: 402
K,^d3>o/oO>[ -> score: 401
K,^d3>o/oP>[ -> score: 400
K,^d3>n/oP>[ -> score: 399
K,^d3>n0oP>[ -> score: 398
K,]d3>n0oP>[ -> score: 399
K,]d3>n/oP>[ -> score: 400
K,]d3>n/oP>[ -> score: 400
K,]d3>n0oP>[ -> score: 399
K,]d3>n1oP>[ -> score: 398
K,]d3?n1oP>[ -> score: 399
K,]d3?n1pP>[ -> score: 400
K

H,\a&Bk6kYET -> score: 403
H,\a&Bk6kYET -> score: 403
H,\a&Bj6kYET -> score: 402
H,\a&Aj6kYET -> score: 401
H,\a&Aj6jYET -> score: 402
H,\a&Aj6jYET -> score: 402
H,\a&Aj6jYET -> score: 402
H,\a&Aj6jYET -> score: 402
H,\a&Aj6jYET -> score: 402
H,\a&Aj6jYET -> score: 402
H+\a&Aj6jYET -> score: 403
H+\a&Aj5jYET -> score: 404
H+\a&Aj5iYET -> score: 405
H+\a&Aj5iYET -> score: 405
H+\a%Aj5iYET -> score: 406
H+\a%Aj5iYET -> score: 406
I+\a%Aj5iYET -> score: 407
I+\a%Aj5iYET -> score: 407
I+\a%Aj5iYFT -> score: 406
H+\a%Aj5iYFT -> score: 405
H+\a%Aj5iYFT -> score: 405
H+\`%Aj5iYFT -> score: 406
H+\`%Aj5iYFT -> score: 406
H+[`%Aj5iYFT -> score: 407
H+[`%Aj5iYFT -> score: 407
H+[`%Aj5iYFU -> score: 406
H+[`%Aj5iYFV -> score: 405
H+[`%Aj5iYFV -> score: 405
H+[`%Aj5iYFV -> score: 405
H+[`%Aj5iXFV -> score: 406
H+\`%Aj5iXFV -> score: 405
H+\`%Aj5iXEV -> score: 406
H+\`%Aj5iXEV -> score: 406
H+\`&Aj5iXEV -> score: 405
H+\`&Bj5iXEV -> score: 406
H+\`&Bj5iXEV -> score: 406
H+]`&Bj5iXEV -> score: 405
H

J4gi,DlBg]I] -> score: 350
I4gi,DlBg]I] -> score: 349
I4gi,DlBg]I] -> score: 349
I4hi,DlBg]I] -> score: 348
I4hi,DlBf]I] -> score: 349
I4hi,DlBf]I] -> score: 349
I4ii,DlBf]I] -> score: 348
I4ii,DlAf]I] -> score: 349
I4ii,DlAf]I] -> score: 349
I4ii,DlAf^I] -> score: 348
I4ii,DlAf_I] -> score: 347
I4ii,DlAf_I] -> score: 347
I4ii,DlAf_I] -> score: 347
I4ii,ClAf_I] -> score: 346
I4ii,ClAf_I] -> score: 346
I4ii,ClAf_I] -> score: 346
I4ii,ClAf_I\ -> score: 347
I4ii,ClAf_I\ -> score: 347
I4ii+ClAf_I\ -> score: 348
I3ii+ClAf_I\ -> score: 349
I3ji+ClAf_I\ -> score: 348
H3ji+ClAf_I\ -> score: 347
H3ki+ClAf_I\ -> score: 346
H3li+ClAf_I\ -> score: 345
H3li+ClAf_I\ -> score: 345
H3li+ClAf_I\ -> score: 345
H3li+CkAf_I\ -> score: 344
H3li+CkAf_I\ -> score: 344
H3li+CkAf`I\ -> score: 343
H3li,CkAf`I\ -> score: 342
H3li,CkAfaI\ -> score: 341
H3li,CkAgaI\ -> score: 340
H3li,CkAgaI[ -> score: 341
H3li-CkAgaI[ -> score: 340
G3li-CkAgaI[ -> score: 341
G3li-CkAgaI[ -> score: 341
G3li-CjAgaI[ -> score: 340
G

H`ll8+\UosUd -> score: 179
H`ll8+\UosUd -> score: 179
H`ll8+\UosUd -> score: 179
H`ll8+\UosUd -> score: 179
H`ll8+\UosUd -> score: 179
H`ll8+\UosVd -> score: 178
H`ll8+\UosVd -> score: 178
H`ll7+\UosVd -> score: 179
H`ll7+\UosVd -> score: 179
H`ll8+\UosVd -> score: 178
H`ll8+\UosVd -> score: 178
H`ll8+\UosVd -> score: 178
H`ll8,\UosVd -> score: 177
H`ll8,\UotVd -> score: 178
H`ll8,\UptVd -> score: 179
H`ll8,\UptVd -> score: 179
H`ll8,\UptVd -> score: 179
H`ll8,\UptVd -> score: 179
H`ll8,\UptVd -> score: 179
H`ll8,\UptVd -> score: 179
Hall8,\UptVd -> score: 178
Gall8,\UptVd -> score: 179
Gall8,[UptVd -> score: 178
Gall8,[UptVd -> score: 178
Gall8,[UotVd -> score: 177
G`ll8,[UotVd -> score: 178
G`ll8,\UotVd -> score: 179
H`ll8,\UotVd -> score: 178
H`ll8,\UotVc -> score: 179
H`ll8,\UotVc -> score: 179
H`ll8,\UotVc -> score: 179
H`ll8,\VotVc -> score: 178
H`ll8,\VotVd -> score: 177
H`ll8,\VotWd -> score: 176
H`ll9,\VotWd -> score: 175
H`ll9,\VotWd -> score: 175
H`ll:,\VotWd -> score: 174
H

Hello,!world -> score: 1
Hello,!world -> score: 1
Hello,!world -> score: 1
Hello,!world -> score: 1
Hello,!world -> score: 1
Hello,!world -> score: 1
Hello, world -> score: 0


In [14]:
print()
print("*************Simulated annealing***************")
print("steps:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


*************Simulated annealing***************
steps:14789, best solution:'Hello, world', best score:0


## 4. Genetic algorithm

This optimization technique is inspired by the theory of evolution. The algorithm starts with a population of random individuals, and selects the ones with the best fitness score (elite). It continues to the next generation with this group. In order to enrich the genetic pool in the current generation, the algorithm adds random mutations and crossover to the elite group. After predefined number of generations, the algorithm returns the top-fit individual. 

### 4.1. Mutation and crossover

In [15]:
# Mutation Operation
def string_mutation(individual):
    i = random.randint(0, len(individual) - 1)

    # mutation changes character at random position to any valid character   
    rchar = chr(random.randint(32, 122))

    individual = individual[0:i] + rchar + individual[(i + 1):]
    return individual


# Mate operation (crossover)
def string_crossover (s1, s2):
    # find the point of crossover
    i = random.randint (0, len(s1)-1)
    return s1[:i]+s2[i:]

### 4.2. Algorithm

Initial population - a list of random strings

In [16]:
def genetic_optimize(population, fitness_function,
                     mutation_function, mate_function,
                     mutation_probability, elite_ratio,
                     maxiterations):

    # How many winners to consider from each generation?
    original_population_size = len(population)
    top_elite = int(elite_ratio * original_population_size)

    # Main loop
    iterations = 0
    for i in range(maxiterations):
        iterations += 1
        individual_scores = [(fitness_function(v), v) for v in population]
        individual_scores.sort()
        ranked_individuals = [v for (s, v) in individual_scores]

        # Start with the pure winners
        population = ranked_individuals[0:top_elite]

        # Add mutated and bred forms of the winners
        while len(population) < original_population_size:
            if random.random() < mutation_probability:
                # Mutation
                c = random.randint(0, top_elite)
                population.append(mutation_function(ranked_individuals[c]))
            else:
                # Crossover
                c1 = random.randint(0, top_elite)
                c2 = random.randint(0, top_elite)
                population.append(mate_function(ranked_individuals[c1], ranked_individuals[c2]))

        # Print current best score
        if print_steps:
            print(individual_scores[0][1]," -> score:", individual_scores[0][0])

        if individual_scores[0][0] == 0:
            return (individual_scores[0][0],individual_scores[0][1], iterations)

    # returns the best solution
    return (individual_scores[0][0], individual_scores[0][1], iterations)



string_population = get_rand_population(2048, len(s), alphabet)
best_score, best_sol, iterations  = genetic_optimize(string_population, string_fitness,
                                string_mutation, string_crossover,
                                mutation_probability=0.25, elite_ratio=0.1,
                                maxiterations=100)
print()
print("*****************GENETIC ALGORITHM ***************")
print("generations:{}, best solution:'{}', best score:{}".format(iterations,best_sol,best_score))


hpnJN4(rx^ig  -> score: 168
<fc_i#"yq^lX  -> score: 88
G_pni#"yq^lX  -> score: 66
L]ynd"#xrvka  -> score: 63
Hgmqi*%jqmnb  -> score: 45
Hgmqn*#xrvkg  -> score: 26
Hgmqn*"yqtka  -> score: 23
Hgmll( wltlh  -> score: 19
Hglnn*#xotlc  -> score: 14
Hglnn*#xotlc  -> score: 14
Hglnn* wotlc  -> score: 10
Hemln*!wotlc  -> score: 8
Hemln* wotlc  -> score: 7
Helln* wotkd  -> score: 6
Helln- wotlc  -> score: 5
Helln- wotld  -> score: 4
Hello- wotld  -> score: 3
Helln- wosld  -> score: 3
Hello- world  -> score: 1
Hello, wosld  -> score: 1
Hello, wosld  -> score: 1
Hello, world  -> score: 0

*****************GENETIC ALGORITHM ***************
generations:22, best solution:'Hello, world', best score:0


### This is the end of the ''Hello, world'' demo.
Copyright &copy; 2022 Marina Barsky